# THIS NOTEBOOK IS NOT TESTED AND COULD HAVE A MAJOR BUG
* Not finnacial advice
* Be careful!
* Notebook is at your own risk!
* I am not responsible for anything in this notebook

In [5]:
from IPython.core.display import HTML
import datetime
import pandas as pd
HTML("""
<style>
.container { width:100% !important; }
</style>
""")


In [6]:
import datetime
import numpy as np
 

### Connect to IB as always

In [7]:
from ib_insync import *
ib = IB()
util.startLoop()
ib.connect(port=4004,clientId=0)

<IB connected to 127.0.0.1:4004 clientId=0>

## Possible Ideas
1. Simple stock trading (Medium Risk, Begginer)
    - Opening Gap Up: 
        * Sell at market or some sort of limit and take profit at gap fill and have stop loss at open + 2 x opening_gap
    - Opening Gap Down:
        * Buy at market or some sort of limit and take profit at gap fill and have wide stop loss at open-2 x opening gap  
2. Selling Options (Less risky, Medium Level)
    - Opening Gap Up: 
        * Sell call option market or some sort of limit and take profit at call price corresponding to gap fill.
    - Opening Gap Down:
        * Sell put option market or some sort of limit and take profit at put price corresponding to gap fill.
3. Trading Spreads (Higher risk, Advanced Level)
    - Opening Gap Up: 
        * Buy bear debit spread market or some sort of limit and take profit at call price at gap fill
    - Opening Gap Down:
        * Buy bull debit spread at market or some sort of limit and take profit at put price at gap fill

### Notes on Gap Fills!

* **Model Dependence on Index Components**: A better index model can depend on the components of the index. A simple model was used for illustration. Stock like MSFT out of wack due to earnings it can screw gap for example 

* **Gap Plays in Individual Stocks**: When individual stocks gap along with the market and there's no specific news driving the individual stock (e.g., if TSLA gaps with S&P by 2% without specific news on TSLA), such movements can be utilized as gap plays.

* **Professional Gaps in High Market Volume**: Gaps occurring during high market volume are termed professional gaps and may take weeks to fill.

* **Fading Strategies Based on Volume**: When there is news with low to moderate premarket volume, these can often be faded regularly.

The above points also incorporate some ideas from John Carter's book, *Mastering the Trade*.

* **Adjust for dividends!**


## Game Plan (Must run between 9:00 and 9:30 PM EST)
####  1. Get keep up to date historical bars with say 5 minutes
#### 2. Attach callback where On first onBarUpdate with hasNewBar = True
    - Calculate Features for Model
    - Make Prediction
    - if Prediction > PROB_THESH + GAP>MIN_GAP_PCT then take trade!

## Review
### 1. Bracket Orders with ib_insync


In [27]:
bracket_orders=ib.bracketOrder(action='BUY',quantity=100,limitPrice=148,takeProfitPrice=150,stopLossPrice=143)
bracket_orders

BracketOrder(parent=LimitOrder(orderId=4552, action='BUY', totalQuantity=100, lmtPrice=148, transmit=False), takeProfit=LimitOrder(orderId=4553, action='SELL', totalQuantity=100, lmtPrice=150, transmit=False, parentId=4552), stopLoss=StopOrder(orderId=4554, action='SELL', totalQuantity=100, auxPrice=143, parentId=4552))

#### Placing Orders

In [29]:
for order in bracket_orders:
    ib.placeOrder(stock,order)

In [34]:
util.df([fill.execution for fill in ib.fills()])

,execId,time,acctNumber,exchange,side,shares,price,permId,clientId,orderId,liquidation,cumQty,avgPrice,orderRef,evRule,evMultiplier,modelCode,lastLiquidity
0,0000e0d5.663c8b81.01.01,2024-05-09 15:24:05+00:00,DU7351505,ISLAND,BOT,100.0,147.98,1822726460,0,4552,0,100.0,147.98,,,0.0,,2


#### Open trades

In [36]:
util.df(ib.openOrders())

,orderId,clientId,permId,action,totalQuantity,orderType,lmtPrice,auxPrice,tif,activeStartTime,...,usePriceMgmtAlgo,duration,postToAts,advancedErrorOverride,manualOrderTime,minTradeQty,minCompeteSize,competeAgainstBestOffset,midOffsetAtWhole,midOffsetAtHalf
0,4553,0,1822726461,SELL,100.0,LMT,150.0,0.0,,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
1,4554,0,1822726462,SELL,100.0,STP,0.0,143.0,,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308


#### Cancelling Orders

In [37]:
for order in ib.openOrders():
    ib.cancelOrder(order)

#### Function to place bracket order

In [191]:
def place_bracket_order(action,qty,limit,take_profit,stop_loss):
    bracket_orders=ib.bracketOrder(action=action,quantity=qty,limitPrice=limit,takeProfitPrice=take_profit,stopLossPrice=stop_loss)
    for order in bracket_orders:
        ib.placeOrder(stock,order)
    

### 2. Requesting Historical Data with keep up to date = True and callbacks

In [8]:
stock = Stock(symbol='ABNB',exchange='SMART',currency='USD')
ib.qualifyContracts(stock)


[Stock(conId=459530964, symbol='ABNB', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='ABNB', tradingClass='NMS')]

In [9]:
bars = ib.reqHistoricalData(stock,"",durationStr='30 D',barSizeSetting='5 mins',whatToShow='TRADES',useRTH=False,keepUpToDate=True)

In [39]:
datetime.datetime.now()

datetime.datetime(2024, 5, 9, 11, 27, 16, 838765)

#### Callbacks
* When bar is completed you actually get 2 bars right after another. One with a new bar with volume = -1

In [40]:
def on_bar_update(bars,hasNewBar):
    if hasNewBar:
        print("has New Bar")
    print(datetime.datetime.now())
    print(util.df(bars).tail(3))
bars.updateEvent.clear()
bars.updateEvent+=on_bar_update

2024-05-09 11:28:16.337377
                          date    open    high     low   close    volume  \
4990 2024-05-09 11:15:00-04:00  147.63  148.25  147.61  148.00  114490.0   
4991 2024-05-09 11:20:00-04:00  148.03  148.20  147.72  147.79   60858.0   
4992 2024-05-09 11:25:00-04:00  147.72  148.28  147.63  148.24   68741.0   

         average  barCount  
4990  147.876947       687  
4991  147.942425       408  
4992  147.952789       298  
2024-05-09 11:28:21.328028
                          date    open    high     low   close    volume  \
4990 2024-05-09 11:15:00-04:00  147.63  148.25  147.61  148.00  114490.0   
4991 2024-05-09 11:20:00-04:00  148.03  148.20  147.72  147.79   60858.0   
4992 2024-05-09 11:25:00-04:00  147.72  148.28  147.63  148.23   68741.0   

         average  barCount  
4990  147.876947       687  
4991  147.942425       408  
4992  147.952789       298  
2024-05-09 11:28:30.638106
                          date    open    high     low   close    volume  \
4

2024-05-09 11:30:11.383845
                          date    open    high     low   close   volume  \
4991 2024-05-09 11:20:00-04:00  148.03  148.20  147.72  147.79  60858.0   
4992 2024-05-09 11:25:00-04:00  147.72  148.28  147.63  148.14  94170.0   
4993 2024-05-09 11:30:00-04:00  148.20  148.21  148.10  148.10   2253.0   

         average  barCount  
4991  147.942425       408  
4992  148.009853       424  
4993  148.160186        20  
2024-05-09 11:30:21.064234
                          date    open    high     low   close   volume  \
4991 2024-05-09 11:20:00-04:00  148.03  148.20  147.72  147.79  60858.0   
4992 2024-05-09 11:25:00-04:00  147.72  148.28  147.63  148.14  94170.0   
4993 2024-05-09 11:30:00-04:00  148.20  148.21  148.10  148.15   2453.0   

         average  barCount  
4991  147.942425       408  
4992  148.009853       424  
4993  148.160171        22  
2024-05-09 11:30:26.320892
                          date    open    high     low   close   volume  \
4991 2024-

In [41]:
bars.updateEvent.clear()

### Get market data for stock

In [20]:
stock_ticker = ib.reqMktData(stock,"",False,False)
ib.sleep(2)

True

In [25]:
stock_ticker.open,stock_ticker.close,stock_ticker.high,stock_ticker.marketPrice()

(146.6, 157.9, 150.01, 147.9)

## Gap Fill with Buying/Shorting Stocks

### Components
### 1. Load Model for predicting if gap will be filled (I am going to assume a dummy model that always predicts the probability as 1)
* Assume just 3 simple features
    * Gap PCT
    * Relative Volume
    * Day_Of_week


#### Fit the Classifier on Historical Data:
* We are using dummy classifier that only predicts 1

In [202]:
features = pd.DataFrame({'GAP_PCT':[0.1,0.3,0.1],'RELATIVE_VOLUME':[1,3,3],'DAY_OF_WEEK':[4,5,5]},index=[0,1,2])
labels=[1,0,1]
features

,GAP_PCT,RELATIVE_VOLUME,DAY_OF_WEEK
0,0.1,1,4
1,0.3,3,5
2,0.1,3,5


In [203]:
from sklearn.dummy import DummyClassifier
model = DummyClassifier(strategy='most_frequent')
model.fit(features,labels)

DummyClassifier(strategy='most_frequent')

In [204]:
model.predict_proba(features)

array([[0., 1.],
       [0., 1.],
       [0., 1.]])

#### Save the model
* Please use a much better model!

In [205]:
from joblib import dump
dump(model,'dummy_model.joblib')

['dummy_model.joblib']

### Assume we fit a model previously and then saved the model

#### Load the model

In [206]:
from joblib import load

model = load('dummy_model.joblib')


### 2. Hyperparams 
 


In [207]:
MIN_GAP_PCT = 0.01 # Min gap % we are willing to make a trade on

STOCK_QTY =100 # Quantity of stock to buy/short

SLIPPAGE_MAX = 5  # Use 0 or negative if want to be careful

STOP_LOSS_TO_TAKE_PROFIT_RATIO = 4 # Ratio of stop loss to take profit. 3 to 1 risk reward is 1/3.. This is a "tight stop" I don't like it (or even I don't like at all)

PROB_THRESH = 0.85 #Probability thresh to make a trade!

assert(STOP_LOSS_TO_TAKE_PROFIT_RATIO>0)
assert(MIN_GAP_PCT>0)

### 3. Contract of Stock we want to trade!
* IF using QQQ/SPY or dividend stocks make sure you adjust for dividends when playing gap fill

In [ ]:
stock = Stock("ABNB",exchange='SMART',currency='USD')
ib.qualifyContracts(stock)

### 4. Request Historical + Market Data 

##### Market Data

In [ ]:
stock_ticker = ib.reqMktData(stock,"",False,False)
ib.sleep(2)

In [209]:
stock_ticker.close,stock_ticker.open,stock_ticker.marketPrice()

(157.9, 146.6, 147.46)

#### Historical Data with Keep Up to Date = True
* Note this doesn't work for 5 second bars! / Live Bars
* Not robust to internet connections. For that you need ib.reqRealTimeBars

In [210]:
bars = ib.reqHistoricalData(stock,"",durationStr='30 D',barSizeSetting='5 mins',whatToShow='TRADES',useRTH=False,keepUpToDate=True)

### Gap Fill Logic for Buying/Shorting Stocks

In [238]:
def gap_trade(bars,ticker):
    i=0
    while True:
        i+=1
        print(i)

        opening_gap = ticker.open - ticker.close
        opening_gap_pct_change = opening_gap/ticker.close
        print(f"Opening gap is {opening_gap}")
        if np.isnan(opening_gap): # Market didn't start yet or at least we didn't get official open price
                print("No open price data yet")
                continue
        else:
            features = compute_features(bars,opening_gap_pct_change)
            
                
        if model.predict_proba(features)[:,1]>=PROB_THRESH: #Model predicts gap will be filled
            print("MODEL PREDICTS GAP WILL BE FILLED!")
            if opening_gap>0: # Gap Up
                print("GAP UP")
                if  (ticker.open-SLIPPAGE_MAX-ticker.close)/ticker.close>=MIN_GAP_PCT: # Is it worth it for you to make a trade?
                    if (ticker.low>ticker.close): # Gap has not yet filled...
                        place_bracket_order(action='SELL',qty=STOCK_QTY,limit=ticker.open-SLIPPAGE_MAX,take_profit=ticker.close,stop_loss=ticker.open+STOP_LOSS_TO_TAKE_PROFIT_RATIO*opening_gap)
                        print("PLACED ORDER")
                        break
                    else:
                        print("GAP ALREADY FILLED")
                        break
                else:
                    print("NO GAP OPPORTUNITY")
                    break
                break

            elif opening_gap<0: # Gap Down
                print("GAP DOWN")
                if  (ticker.open+SLIPPAGE_MAX-ticker.close)/ticker.close<=-MIN_GAP_PCT: # Is it worth it for you to make a trade?
                    if (ticker.high<ticker.close):   # Gap not yet filled
                        place_bracket_order(action='BUY',qty=STOCK_QTY,limit=ticker.open+SLIPPAGE_MAX,take_profit=ticker.close,stop_loss=ticker.open+STOP_LOSS_TO_TAKE_PROFIT_RATIO*opening_gap)
                        print("PLACED ORDER")
                        break
                    else:
                        print("GAP ALREADY FILLED!")
                        break

                else:
                    print("NO GAP OPPORTUNITY")
                    break

                break
            else:
                print("NO GAP OPPORTUNITY")
                break
        else: 
            print("MODEL PREDICTS GAP WILL NOT BE FILLED!")
            break
            


### 5. Define callback for opening gap
* Must run before market open!

In [239]:
market_open = datetime.time(9,30)

market_open = datetime.time(13,40)


def on_bar_update(bars,hasNewBar):
    if hasNewBar:
        if bars[-1].date.time()==market_open: #Check if market is open!
            print(f"Volume is {bars[-1].volume}")
            gap_trade(bars[:-1],stock_ticker) # until index -1 cuz that is latest completed bar...
            bars.updateEvent.clear()
            print("DONE RUNNING GAP FILL STRATEGY!")
    
        
    
bars.updateEvent.clear()
bars.updateEvent+=on_bar_update

Volume is -1.0
1
Opening gap is -11.300000000000011
MODEL PREDICTS GAP WILL BE FILLED!
GAP DOWN
PLACED ORDER
DONE RUNNING GAP FILL STRATEGY!


In [223]:
def compute_features(bars,opening_gap_pct):
    df = util.df(bars)
    df['day_of_week']=df.date.dt.day_of_week
    df['day']=df.date.dt.date
    PREMARKET=df.set_index('date').between_time('4:00','9:25').groupby('day').agg(volume=('volume','sum'))
    PREMARKET['volume_14_ma']=PREMARKET.rolling(14)['volume'].mean()

    # Take last Entry of each feature...
    day_of_week = df['day_of_week'].iloc[-1]
    relative_volume = PREMARKET.volume.iloc[-1]/PREMARKET.volume_14_ma.iloc[-1]
    
    
    return [[opening_gap_pct ,relative_volume,day_of_week]]


In [230]:
util.df(bars).date.dt.date

0       2024-03-28
1       2024-03-28
2       2024-03-28
3       2024-03-28
4       2024-03-28
           ...    
5013    2024-05-09
5014    2024-05-09
5015    2024-05-09
5016    2024-05-09
5017    2024-05-09
Name: date, Length: 5018, dtype: object

Opening gap is -11.300000000000011
MODEL PREDICTS GAP WILL BE FILLED!
GAP DOWN
PLACED ORDER
DONE RUNNING GAP FILL STRATEGY!
Volume is -1.0
Opening gap is -11.300000000000011
MODEL PREDICTS GAP WILL BE FILLED!
GAP DOWN
PLACED ORDER
DONE RUNNING GAP FILL STRATEGY!
Volume is -1.0
Opening gap is -11.300000000000011
MODEL PREDICTS GAP WILL BE FILLED!
GAP DOWN
PLACED ORDER
DONE RUNNING GAP FILL STRATEGY!


In [240]:
util.df([f.execution for f in ib.fills()])

,execId,time,acctNumber,exchange,side,shares,price,permId,clientId,orderId,liquidation,cumQty,avgPrice,orderRef,evRule,evMultiplier,modelCode,lastLiquidity
0,0000e0d5.663c8b81.01.01,2024-05-09 15:24:05+00:00,DU7351505,ISLAND,BOT,100.0,147.98,1822726460,0,4552,0,100.0,147.98,,,0.0,,2
1,0000e0d5.663ca27b.01.01,2024-05-09 17:00:23+00:00,DU7351505,EDGEA,BOT,100.0,147.62,1822726463,0,4564,0,100.0,147.62,,,0.0,,2
2,0000e0d5.663ca27d.01.01,2024-05-09 17:00:27+00:00,DU7351505,CHX,SLD,100.0,147.46,1822726465,0,4566,0,100.0,147.46,,,0.0,,2
3,0000dc8f.663c5881.01.01,2024-05-09 17:02:15+00:00,DU7351505,IBKRATS,BOT,100.0,147.36,1822726466,0,4567,0,100.0,147.36,,,0.0,,2
4,0000dc8f.663c5964.01.01,2024-05-09 17:35:01+00:00,DU7351505,IBKRATS,BOT,100.0,147.63,1822726469,0,4571,0,100.0,147.63,,,0.0,,2
5,0000e0d5.663cabd2.01.01,2024-05-09 17:35:02+00:00,DU7351505,ISLAND,BOT,100.0,147.63,1822726472,0,4574,0,100.0,147.63,,,0.0,,2
6,0000e0d5.663cabd7.01.01,2024-05-09 17:35:03+00:00,DU7351505,ISLAND,BOT,100.0,147.63,1822726475,0,4577,0,100.0,147.63,,,0.0,,2
7,0000dc8f.663c598f.01.01,2024-05-09 17:40:02+00:00,DU7351505,IBKRATS,BOT,100.0,147.58,1822726478,0,4580,0,100.0,147.58,,,0.0,,2


In [241]:
util.df(ib.openOrders())

,orderId,clientId,permId,action,totalQuantity,orderType,lmtPrice,auxPrice,tif,activeStartTime,...,usePriceMgmtAlgo,duration,postToAts,advancedErrorOverride,manualOrderTime,minTradeQty,minCompeteSize,competeAgainstBestOffset,midOffsetAtWhole,midOffsetAtHalf
0,4581,0,1822726479,SELL,100.0,LMT,157.9,0.0,,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
1,4582,0,1822726480,SELL,100.0,STP,0.0,101.4,,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308


Peer closed connection.


In [236]:
for order in ib.openOrders():
    ib.cancelOrder(order)